# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingeniería en sistemas** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 05**:

**Fecha**: 21/02/2024

**Nombre del equipo**: Equipo
 
**Profesor**: Pablo Camarillo Ramirez

In [ ]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Transformations-Actions") \
    .master("spark://82f3da55ae59:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/22 22:41:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
import importlib
import team_name.spark_utils

importlib.reload(team_name.spark_utils)

columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = team_name.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

In [7]:
filtered_popularity_df = tiktok_df.select("popularity").filter(tiktok_df["popularity"] > 80)

In [8]:
row_count = filtered_popularity_df.count()
row_count

1023

In [9]:
group_by_genre = tiktok_df.select("genre", "duration").groupBy("genre").avg("duration")
group_by_genre.show()

+------------------+------------------+
|             genre|     avg(duration)|
+------------------+------------------+
|TIKTOK PHILIPPINES|196807.97061442563|
|      TIKTOK DANCE|180901.24283501162|
|           _TIKTOK|195071.78653012996|
|        TIKTOK OPM|255431.57171314742|
+------------------+------------------+



In [18]:
top_energetic = tiktok_df. \
select("genre", "energy"). \
orderBy("energy", ascending=False)
top_energetic.limit(10).show()

+------------------+------------------+
|             genre|            energy|
+------------------+------------------+
|        TIKTOK OPM|0.9990000000000001|
|           _TIKTOK|             0.998|
|           _TIKTOK|             0.995|
|           _TIKTOK|             0.995|
|           _TIKTOK|             0.995|
|           _TIKTOK|0.9940000000000001|
|TIKTOK PHILIPPINES|             0.993|
|           _TIKTOK|             0.992|
|TIKTOK PHILIPPINES|             0.992|
|      TIKTOK DANCE|             0.992|
+------------------+------------------+



In [20]:
duration_by_playlist = tiktok_df. \
select("playlist_name", "duration_mins"). \
groupBy("playlist_name").sum("duration_mins")
duration_by_playlist.show()

+--------------------+------------------+
|       playlist_name|sum(duration_mins)|
+--------------------+------------------+
|5IZc3KIVFhjzJ0L2k...| 7.474666666666667|
|08ia51KbTcfs4QVT5...|            4.1485|
|7xVLFuuYdAvcTfcP3...| 9.456433333333333|
|2RBILNmyq8p4fqVWO...| 2.162933333333333|
|6GdDjthxbTGBV9rl2...|3.3209166666666667|
|7krYEnB1OI1RbnJBa...|2.0957666666666666|
|1FgPyHX7HruKDL4Tx...|            2.4448|
|62RtxFf9epYNWOUHJ...|2.6694333333333335|
|5ow0sNF1zSqp71Ix5...|2.7334833333333335|
|0LlJbV4lyzJYE14YC...|10.709133333333334|
|6NFKf8vBApSvtzkap...|3.7074333333333334|
|5P8lyudWE7HQxb4lu...| 4.250666666666667|
|2BgEsaKNfHUdlh97K...| 3.116433333333333|
|7F9vK8hNFMml4GtHs...| 3.173783333333333|
|4vVTI94F9uJ8lHNDW...|3.3657666666666666|
|2uULRpRtKhCdojXwo...|               2.2|
|1tRlGMHsf21FDo6pj...|           1.79005|
|215fAfwkWtlj30ofd...|2.3214166666666665|
|3bidbhpOYeV4knp8A...|            4.3057|
|0YFocHKmrMme7Isel...| 4.810383333333333|
+--------------------+------------